En las secciones anteriores exploramos cómo "piensan" los modelos generativos como GPT (usando la librería `tiktoken`). Ahora vamos a cambiar de enfoque para estudiar **BERT** (Bidirectional Encoder Representations from Transformers), uno de los modelos más influyentes para tareas de comprensión y clasificación.

### ¿En qué se diferencia el Tokenizador de BERT?

A diferencia de GPT, que lee de izquierda a derecha para predecir la siguiente palabra, BERT lee toda la frase a la vez (izquierda y derecha) para entender el contexto profundo. Por eso, su tokenización tiene requisitos especiales:

1.  **Librería Diferente:** Usaremos la famosa librería `transformers` de Hugging Face, el estándar actual en la industria.
2.  **Tokens Especiales:** BERT necesita "pistas" estructurales que GPT no usa obligatoriamente:
    * `[CLS]` (Classification): Un token que siempre va al inicio y resume el significado de toda la frase.
    * `[SEP]` (Separator): Un token que marca el final de una oración o separa dos oraciones distintas.
    * `[PAD]` (Padding): Relleno para que todas las secuencias tengan la misma longitud.
3.  **Vocabulario:** Usaremos el modelo `bert-base-uncased`, que convierte todo a minúsculas antes de procesar (simplificando el vocabulario).

¡Vamos a importar la librería y ver estas diferencias en acción!

# Ejercicio Práctico: Tokenización con BERT

### 1. Importar y Cargar el Tokenizador
A diferencia de GPT (que usa `tiktoken`), para BERT utilizamos la famosa librería `transformers` de Hugging Face.
Cargaremos el modelo `bert-base-uncased`.
* **bert-base:** La versión estándar.
* **uncased:** Convierte todo a minúsculas (ignora mayúsculas).

In [3]:
# Importar el tokenizador de BERT
from transformers import BertTokenizer

# Cargar el tokenizador pre-entrenado
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

### 2. Inspeccionar el Tokenizador
Vamos a ver qué métodos y atributos tiene disponibles esta herramienta usando `dir()`.

In [4]:
# Inspeccionar la información del objeto tokenizer
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_added_tokens_decoder',
 '_added_tokens_encoder',
 '_auto_class',
 '_batch_encode_plus',
 '_batch_prepare_for_model',
 '_call_one',
 '_convert_id_to_token',
 '_convert_token_to_id',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_pa

### 3. Explorando el Vocabulario
Al igual que antes, BERT tiene un vocabulario fijo. Vamos a ver:
1.  Una muestra de tokens (del 20,000 al 20,100).
2.  El tamaño total del vocabulario.
3.  El ID específico para la palabra "science".

In [7]:
# Obtener todos los tokens del vocabulario
all_tokens = list(tokenizer.get_vocab().keys())

# Imprimir un rango específico
all_tokens[20000:20100]

# Tamaño del vocabulario e ID de una palabra específica
print(tokenizer.vocab_size)
tokenizer.get_vocab()['science']

30522


2671

### 4. Tokenizando una Palabra (Manualmente)
Podemos convertir una palabra en su ID numérico de dos formas:
1.  Usando el método `convert_tokens_to_ids`.
2.  Buscándola directamente en el diccionario `vocab`.

In [9]:
# Tokenizando una palabra
word = 'science'

res1 = tokenizer.convert_tokens_to_ids(word)
res2 = tokenizer.get_vocab()[word]

print(res1)
print(res2)

2671
2671


### 5. Intentando Codificar Texto (El Error Común)
Si intentamos usar `convert_tokens_to_ids` o buscar en el diccionario una **frase entera**, fallará.
¿Por qué? Porque el diccionario contiene *palabras* (o sub-palabras), no frases completas. La frase "science is great" no es una llave del diccionario.

In [10]:
# Codificando un texto (Esto dará error o no funcionará como esperamos)
text = 'science is great'

# Esto fallará porque 'convert_tokens_to_ids' espera una lista de tokens, no un string
# y el diccionario espera una palabra exacta.
# res1 = tokenizer.convert_tokens_to_ids(text)
# res2 = tokenizer.get_vocab()[text]

# print(res1)
# print(res2)

### 6. La Forma Correcta: `tokenizer.encode`
El método `encode` es el inteligente. Hace todo el trabajo:
1.  Separa la frase.
2.  Busca los IDs.
3.  **Importante:** Agrega tokens especiales que BERT necesita:
    * `[CLS]`: Inicio de clasificación (siempre va al principio).
    * `[SEP]`: Separador (siempre va al final de la frase).

In [11]:
# El método correcto: encode
res3 = tokenizer.encode(text)

for i in res3:
  print(f'Token {i} is "{tokenizer.decode(i)}"')

# [CLS] = classification (token de inicio)
# [SEP] = sentence separation (token de final)

print('')
# Decodificar saltando los especiales (texto limpio)
print(tokenizer.decode(res3, skip_special_tokens=True))
# Decodificar mostrando todo (texto crudo que ve el modelo)
print(tokenizer.decode(res3, skip_special_tokens=False))

Token 101 is "[CLS]"
Token 2671 is "science"
Token 2003 is "is"
Token 2307 is "great"
Token 102 is "[SEP]"

science is great
[CLS] science is great [SEP]


### 7. El Problema de la Recursividad
Si codificamos, decodificamos y volvemos a codificar, BERT seguirá agregando tokens `[CLS]` y `[SEP]` una y otra vez. ¡Cuidado con esto!

In [13]:
# BERT agrega [CLS]...[SEP] con cada codificación
tokenizer.decode(tokenizer.encode(tokenizer.decode(tokenizer.encode( text ))))

'[CLS] [CLS] science is great [SEP] [SEP]'

### 8. Llamada Directa (Para uso en Producción)
En aplicaciones reales, solemos llamar al tokenizador directamente como una función.
Esto devuelve un diccionario con:
* `input_ids`: Los números de los tokens.
* `token_type_ids`: Para diferenciar frase A de frase B.
* `attention_mask`: Para decir qué es texto y qué es relleno (padding).

### 9. Profundizando: Tokenize vs Encode
Aquí veremos la diferencia clave paso a paso:
1.  `tokenize()`: Solo corta el texto en pedazos (palabras/subpalabras). **No agrega** `[CLS]` ni `[SEP]`.
2.  `encode()`: Corta el texto, convierte a números **Y agrega** `[CLS]` y `[SEP]`.

Observa la diferencia en los resultados impresos al final.

In [14]:
# Llamando a la clase directamente
tokenizer(text)

{'input_ids': [101, 2671, 2003, 2307, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [15]:
# Más sobre tokenización
sentence = 'AI is both exciting and terrifying.'

print('Original sentence:')
print(f'  {sentence}\n')

# 1. Segmentar el texto en tokens (sin convertir a números todavía)
tokenized = tokenizer.tokenize(sentence)
print('Tokenized (segmented) sentence:')
print(f'  {tokenized}')

# 2. Convertir esos tokens segmentados a IDs numéricos
ids_from_tokens = tokenizer.convert_tokens_to_ids(tokenized)
print(f'  {ids_from_tokens}\n')

# 3. Y finalmente, usar 'encode' directamente desde el texto original
# Nota: encode agrega automáticamente 101 ([CLS]) y 102 ([SEP])
encodedText = tokenizer.encode(sentence)
print('Encoded from the original text:')
print(f'  {encodedText}\n\n')

# Ahora decodificamos para comparar
print('Decoded from token-wise encoding (sin especiales):')
print(f'  {tokenizer.decode(ids_from_tokens)}\n')

print('Decoded from text encoding (con especiales):')
print(f'  {tokenizer.decode(encodedText)}')

Original sentence:
  AI is both exciting and terrifying.

Tokenized (segmented) sentence:
  ['ai', 'is', 'both', 'exciting', 'and', 'terrifying', '.']
  [9932, 2003, 2119, 10990, 1998, 17082, 1012]

Encoded from the original text:
  [101, 9932, 2003, 2119, 10990, 1998, 17082, 1012, 102]


Decoded from token-wise encoding (sin especiales):
  ai is both exciting and terrifying.

Decoded from text encoding (con especiales):
  [CLS] ai is both exciting and terrifying. [SEP]


### Entendiendo la "Attention Mask" (Máscara de Atención)

Cuando llamamos a `tokenizer(text)`, vemos que devuelve algo llamado `attention_mask`: `[1, 1, 1, 1, 1]`.

**¿Qué significan estos unos y ceros?**

Los modelos como BERT necesitan recibir entradas de longitud fija (por ejemplo, siempre 512 tokens).
* Si tu frase es corta (5 palabras), el resto se rellena con tokens vacíos (`[PAD]`).
* La **máscara** es un mapa binario para que el modelo sepa qué leer y qué ignorar.

#### La Lógica Binaria:
* **`1` (Atención):** "¡Oye BERT, esto es importante!". Representa un token real. Puede ser una palabra completa, una sílaba de una palabra larga, o un token especial (`[CLS]`, `[SEP]`). El modelo **no ignora** nada que tenga un 1; procesa su significado y contexto.
* **`0` (Ignorar):** "Esto es relleno, no lo mires". Representa el token de padding (`[PAD]`). Matemáticamente, el modelo multiplica por 0 la atención hacia estos tokens para que no afecten el resultado final.

#### Ejemplo con Padding (Relleno):
Imagina que configuramos el tokenizador para una longitud fija de 10:

```python
# Frase real: "Science is great" (incluyendo [CLS] y [SEP] son 5 tokens)
# Longitud forzada: 10
# Resultado:
# Tokens: [101, 2671, 2003, 2307, 102, 0, 0, 0, 0, 0]
# Mask:   [  1,    1,    1,    1,   1, 0, 0, 0, 0, 0]

Nota sobre palabras largas: Si tienes una palabra muy larga que se divide en sub-tokens (ej: token + ##ization), ambos recibirán un 1. La máscara no juzga la calidad de la palabra, solo te dice: "Aquí hay contenido, procésalo".